In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

In [3]:
def read_data(path):
    f = open(path,'r')
    text = f.read()
    examples = [example.split(' ') for example in text.split('\n')[:-1]]
    labels = [int(line[0]) for line in examples]
    data = [line[1:] for line in examples]
    return data,np.array(labels).reshape((-1,1))

In [4]:
train_data, train_labels = read_data('data/train.txt')
dev_data, dev_labels = read_data('data/dev.txt')
test_data, test_labels = read_data('data/test.txt')

In [5]:
train_labels = torch.tensor(train_labels)
dev_labels = torch.tensor(dev_labels)
test_labels = torch.tensor(test_labels)

In [6]:
def create_vocab(data):
    flatten = [w for line in data for w in line]
    unique = list(set(flatten))
    word2idx = {word: idx for idx,word in enumerate(unique)}
    return unique,word2idx

In [7]:
def create_bag_of_words(data, word2idx=None):
    if word2idx is None:
        raise Error('create_bag_of_words need a word2idx mapping!')
    bag_of_words = np.zeros((len(data), len(word2idx)))
    for line in range(len(data)):
        for word in data[line]:
            if word in word2idx:
                bag_of_words[line][word2idx[word]] += 1
    return bag_of_words

In [14]:
class BagOfWordsClassifier(nn.Module):

    # Initialize the classifier
    def __init__(self, num_labels, vocab_size):
        super(BagOfWordsClassifier, self).__init__()

        self.linear = nn.Linear(vocab_size, num_labels)
        self.vocab_size = vocab_size
        self.num_labels = num_labels
        
    def forward(self, bow_vec):
        # Pass the input through the linear layer,
        # then pass that through log_softmax.
        # Many non-linearities and other functions are in torch.nn.functional
        z1 = self.linear(bow_vec)
        return torch.sigmoid(z1)

In [15]:
train_vocab, train_word2idx = create_vocab(train_data)
model = BagOfWordsClassifier(2, len(train_vocab))

In [16]:
## Get log probs before training
with torch.no_grad():
    for data, label in zip(dev_data,dev_labels):
        bow_vec = create_bag_of_words([data], word2idx=train_word2idx)
        bow_vec = torch.tensor(bow_vec).float()
        log_probs = model(bow_vec)

In [17]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [56]:
bow_vectors = create_bag_of_words(train_data, word2idx=train_word2idx)
bow_vectors = bow_vectors.reshape((-1,1,len(train_word2idx)))

In [79]:
## Train the model
for epoch in tqdm(range(5)):    
    for bow_vec, label in zip(bow_vectors, train_labels):
        model.zero_grad()
        
        bow_vec = torch.tensor(bow_vec).float()
        log_probs = model(bow_vec)
        
        loss = loss_function(log_probs, label)
        loss.backward()
        optimizer.step()




  0%|          | 0/5 [00:00<?, ?it/s]


 20%|██        | 1/5 [00:09<00:38,  9.60s/it]


 40%|████      | 2/5 [00:18<00:28,  9.51s/it]


 60%|██████    | 3/5 [00:29<00:19,  9.73s/it]


 80%|████████  | 4/5 [00:41<00:10, 10.45s/it]


100%|██████████| 5/5 [00:52<00:00, 10.67s/it]




In [80]:
## Test on dev set
dev_bow_vecs = create_bag_of_words(dev_data, word2idx=train_word2idx)
dev_bow_vecs = dev_bow_vecs.reshape((-1,1,len(train_word2idx)))
acc_count = 0
for bow_vec, label in zip(dev_bow_vecs, dev_labels):        
        bow_vec = torch.tensor(bow_vec).float()
        log_probs = model(bow_vec)
        print(log_probs)

tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9996]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9996]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9996]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9992, 0.9927]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], g

tensor([[0.9999, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9995]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9992, 0.9853]], grad_fn=<SigmoidBackward>)
tensor([[0.9990, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9995, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9996, 0.9999]], g

tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9992]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9994]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], g

tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9995]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9971, 0.9992]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], g

tensor([[0.9992, 0.9992]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9992, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9996]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9967, 0.9989]], grad_fn=<SigmoidBackward>)
tensor([[0.9973, 0.9993]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9995]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9990, 0.9994]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9995]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], g

...

tensor([[0.9974, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9961, 0.9993]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9978, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9976, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[0.9974, 0.9996]], grad_fn=<SigmoidBackward>)
tensor([[0.9979, 0.9995]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9991, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9992, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9979, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], g

tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9997, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9995]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9997, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], g

tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9991, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9997, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9996]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9993, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9984, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[0.9995, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9975, 0.9994]], grad_fn=<SigmoidBackward>)
tensor([[0.9995, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9980, 0.9996]], grad_fn=<SigmoidBackward>)
tensor([[0.9991, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9971, 0.9993]], g

tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9989, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9997, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9991, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9984, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9996, 0.9996]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9992, 1.0000]], g

tensor([[0.9996, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9997, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9995, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9988, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[0.9969, 0.9993]], grad_fn=<SigmoidBackward>)
tensor([[0.9990, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9986, 0.9996]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], g

In [81]:
## Test on dev set
dev_bow_vecs = create_bag_of_words(train_data, word2idx=train_word2idx)
dev_bow_vecs = dev_bow_vecs.reshape((-1,1,len(train_word2idx)))
acc_count = 0
for bow_vec, label in zip(dev_bow_vecs, train_labels):        
        bow_vec = torch.tensor(bow_vec).float()
        log_probs = model(bow_vec)
        print(log_probs)

tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9984, 0.9902]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9984, 0.9902]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], g

tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9996, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], g

tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9995]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9994]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9997, 0.9995]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], g

tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9974]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], g

tensor([[0.9999, 0.9994]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9997, 0.9994]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[0.9990, 0.9996]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9996]], g

...

tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9989, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9974, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], g

tensor([[0.9958, 0.9988]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9996]], grad_fn=<SigmoidBackward>)
tensor([[0.9994, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[0.9983, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[0.9986, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9945, 0.9995]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], g

tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9989, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9997, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9987, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9967, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[0.9972, 0.9996]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9997, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9995, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9996]], grad_fn=<SigmoidBackward>)
tensor([[0.9990, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9972, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[0.9993, 0.9999]], g

tensor([[0.9999, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9975, 0.9995]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9949, 0.9994]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9979, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], g

tensor([[0.9998, 0.9996]], grad_fn=<SigmoidBackward>)
tensor([[0.9963, 0.9994]], grad_fn=<SigmoidBackward>)
tensor([[0.9967, 0.9995]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9998]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9965, 0.9993]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9997]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[0.9999, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], grad_fn=<SigmoidBackward>)
tensor([[0.9998, 0.9999]], grad_fn=<SigmoidBackward>)
tensor([[1.0000, 1.0000]], g